# Building and Training a CNN
Today we're going to take the datasets we created yesterday and train a convolutional neural network (CNN) on them. Since this directly depends on the output files from yesterday, take a moment to copy and paste your output files into today's directory. Again, let's take a second to remember our two guiding questions:

- _What_ is in an image (e.g. debris, buildings, etc.)?
- _Where_ are these things located _in 3D space_ ?

Recall that we are posing the first question as a classification problem (e.g. does this image have flooding or not), and we are deploying a CNN to aid in the classification.

As before, this is heavily based off of the following tutorials: https://github.com/LADI-Dataset/ladi-tutorial/blob/master/Tutorials/Pytorch_Data_Load.md, 

https://github.com/LADI-Dataset/ladi-tutorial/blob/master/Tutorials/Train_Test_Classifier.md

## PyTorch
Yesterday we developed simple machine learning models using sci-kit learn. While the package is good for quick and dirty machine learning models, we are going to need much more powerful tools to take on an image classification task of the kind we're seeing. To that end, we're going to be working with PyTorch, typically considered as the state of the art in deep learning. It was developed primarily by Facebook AI in and released for Python in 2017. There are two main advantages of PyTorch vs. many other Python packages:
- A fairly high level interface for building neural networks
- Parallelization support via GPU
While it is one of the easier ways to build a neural network, there is still a fairly steep learning curve. Thus, today will be all about learning how to implement in PyTorch (and less about what's going on under the hood). 

In [ ]:
import torch
import pandas as pd
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
import pathlib

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter

from PIL import Image
import datetime

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms, utils

import warnings
warnings.filterwarnings('ignore') # turn off warnings
# warnings.filterwarnings('default') # regular level warnings

In [ ]:
torch.__version__

## Creating a Dataset from your csv files
The first step to using PyTorch is creating a dataset. All datasets in PyTorch need to be a subclass of torch.utils.data.Dataset, which is the general class that represents a dataset. When creating your dataset, you usually need to overwrite two methods:
- ```__len__```, so that when you call len(dataset) it returns the number of images
- ```__getitem__```, so that when you call dataset[i] it returns the ith item. 
As usual, we're also going to define ```__init__``` to take in the arguments we care about.

We also need to decide what we want a sample of our dataset to look like. Ours is going to look like a dictionary as follows: ```{'image': image, 'image_name': img_name, 'damage:flood/water': label, uuid': uuid, 'timestamp': timestamp, 'gps_lat': gps_lat, 'gps_lon': gps_lon, 'gps_alt': gps_alt, 'orig_file_size': file_size, 'orig_width': width, 'orig_height': height}```. Of course, there's no requirement that it be a dictionary, though it should somehow incorporate the input and output (in our case, the image and the label). 

In [ ]:
# make sure GPU is available
torch.cuda.is_available()

In [ ]:
# convenient function for showing the images
def show_image(image):
    plt.imshow(image)
    # pause a bit so that plots are updated
    plt.pause(0.01)

In [ ]:
class LadiDataset(Dataset):
    def __init__(self, label_csv, class_names, transform = None):
        """
        Args:
            label_csv (str): path to csv with columns for url, local_path, and columns for each label class
            class_names (list[str]): list of the column names corresponding to each label class
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.label_data_df = pd.read_csv(label_csv)
        self.class_names = class_names
        self.transform = transform
        
    def __len__(self):
        return len(self.label_data_df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        ## Load images from local directory. There is no need to redownload images to local machine. ##
        local_path = self.label_data_df.iloc[idx]['local_path']
        url = self.label_data_df.iloc[idx]['url']
        try:
            image = Image.fromarray(io.imread(local_path))
            img_name = local_path
        except:
            image = Image.fromarray(io.imread(url))
            img_name = url
        label = torch.tensor(self.label_data_df.iloc[idx].loc[self.class_names].values.astype(float))
        
        if self.transform:
            image = self.transform(image)

        example = {'image': image, 
                   'image_name': img_name, 
                   'label': label}

        return example

In [ ]:
classes = ['damage:flood/water','damage:rubble','damage:misc']
# Let's do a quick sanity check
damage_dataset = LadiDataset('damage_examples.csv', classes)

fig = plt.figure()

for i, sample in enumerate(damage_dataset):

    print(i, 
          sample['label'], 
          sample['image_name'])

    plt.tight_layout()
    plt.title('Sample #{}'.format(i))
    show_image(sample['image'])

    if i == 5:
        plt.show()
        break

## Transformations
There are now two things to take care of:
- Neural networks usually expect all images to be of the same size, but sometimes datasets are not all of the same size. The LADI dataset includes image of different sizes. 
- Like we saw in our thought exercise last class, there is a huge amount of variability. Because of this, the top performing CNN's usually train on hundreds of thousands of images, much more than what we have! We're going to have to somehow increase the number of images available to us. 

One solution is to augment the dataset by *transforming* it. Most transformations are actually fairly straightforward.

- ```Resize```: to resize the input PIL Image to the given size.
- ```RandomCrop```: to crop from image randomly. This is data augmentation.
- ```RandomRotation```: to rotate the image by angle.
- ```RandomHorizontalFlip```: to horizontally flip the given PIL Image randomly with a given probability.
- ```ToTensor```: to convert the numpy images to torch images (we need to swap axes).

These functions each have their own set of arguments, which can be referenced here(https://pytorch.org/docs/stable/torchvision/transforms.html)

Now let's try to apply these transforms, first individually and then all at once. The key to effective transforms is to have "enough" transformation that the input image is visually distinct from the output image, but not so much that the output image is no longer representative of the dataset. 

In [ ]:
flip = transforms.RandomHorizontalFlip(p=0.5) # some probability of flipping
scale = transforms.Resize(1024) # resize to fixed size
affine = transforms.RandomAffine(degrees=20, translate=(0.15, 0.15), scale=(0.8,1.6)) # affine transform
jitter = transforms.ColorJitter(brightness=0.4, contrast=0.3, saturation=0.3, hue=0.02) # add color jitter noise
perspective = transforms.RandomPerspective(0.3, 0.5) # random perspective changes
c_crop = transforms.CenterCrop(512) # crop center 512x512 pixels

flip_demo = transforms.RandomHorizontalFlip(1) # flip with 100% chance just to demo
composed_demo = transforms.Compose([scale,
                               affine,
                               perspective,
                               jitter,
                               c_crop,
                              flip_demo])

# Apply each of the above transforms on sample.

sample = damage_dataset[190]
for i, tsfrm in enumerate([scale, c_crop, affine, flip_demo, jitter, perspective, composed_demo]):
    transformed_image = tsfrm(sample['image'])
    fig, ax = plt.subplots(1, 1, figsize=(5,5))
    plt.tight_layout()
    ax.set_title(type(tsfrm).__name__)
    show_image(transformed_image)

plt.show()

## DataLoader
Compared to simple ```for``` loop to iterate over data, ```torch.utils.data.DataLoader``` is an iterator which provides more features:

- Batching the data.
- Shuffling the data.
- Load the data in parallel using multiprocessing workers.

In the previous section, three transforms are performed on a sample. In this section, users can learn to use ```DataLoader``` to transform all images in the dataset.

First, a new dataset with transform needs to be defined.

In [ ]:
# for training, we're adding the convert to tensor transform
composed = transforms.Compose([scale,
                               affine,
                               perspective,
                               jitter,
                               c_crop,
                              flip,
                              transforms.ToTensor()])
transformed_dataset = LadiDataset('damage_examples.csv', classes, composed)

In [ ]:
# create a dataloader to load the data for illustrative purposes
dataloader = DataLoader(transformed_dataset, batch_size=8,
                        shuffle=True, num_workers=4)

In [ ]:
# Helper function to show a batch
def show_images_batch(sample_batched):
    images_batch = sample_batched['image']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)
    grid_border_size = 2

    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

    for i in range(batch_size):
        plt.title('Batch from dataloader')

In [ ]:
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['image'].cuda().size())

    # observe 1st batch and stop.
    if i_batch == 0:
        plt.figure()
        show_images_batch(sample_batched)
        plt.axis('off')
        plt.ioff()
        plt.show()
        break

### Exercise
Using the csv files you created yesterday, create your own DataLoader using transforms that you think would be interesting or helpful. Here's a full list of transforms: https://pytorch.org/docs/stable/torchvision/transforms.html

As a group, answer the following questions
- Are the image labels accurate? That is to say, if it is labeled as having a flood, does it actually have a flood?
- Do the transforms add enough variety to the dataset? Do they still look representative of the dataset?
- Do the transforms distort from the labels? For example, if you use a random crop, is the thing that is being labeled still present in the image?

## Training a neural network
We are now ready to train a CNN! We will be using a pre-trained model as our starting point to reduce the amount of training that we need to do. The `torchvision` module includes common pretrained models. We will be using one called `resnet50`

In [ ]:
torch.backends.cudnn.benchmark = True # flag for some GPU optimizations
torch.hub.set_dir('pytorch-hub-cache') # set location for saving pretrained models
net = torchvision.models.get_model('resnet50', pretrained=True) # load pretrained model

# pretrained neural network is missing the last layer, we add it back on
net.fc = nn.Linear(2048, 3) # resnet50 create 2048 features in its first half, we have 3 classes
net = nn.DataParallel(net) # specify the data distribution strategy if we have more than one GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # send it to GPU if it's available
net.to(device) 
print("ResNet ready")

We are now going to take a familiar step: we're going to split the dataset into a training and testing set.

In [ ]:
def get_train_test_loaders(dataset, test_split_ratio=0.2, batch_size=4, shuffle_dataset=True, random_seed=0, num_workers=1):

    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(test_split_ratio * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, test_indices = indices[split:], indices[:split]

    # Creating data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    test_sampler = SubsetRandomSampler(test_indices)

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                               sampler=train_sampler, num_workers=num_workers)
    test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                               sampler=test_sampler, num_workers=num_workers)
    return train_loader, test_loader

In [ ]:
damage_train_loader, damage_test_loader = get_train_test_loaders(transformed_dataset, batch_size=8, random_seed=42, num_workers=4)

We're now going to define the loss function and the optimizer. You can think of the loss function as the amount of error you want to minimize, while the optimizer is the specific tool you're going to use to minimize that loss. 

Because the data is not evenly represented, we want to re-weight each class based on how many examples there are in the dataset. This way we don't get biased toward more commonly represented classes in the data

In [ ]:
damage_df = pd.read_csv('damage_examples.csv')

# parameters for Damage Model
flood_weight = (len(damage_df)-damage_df['damage:flood/water'].sum())/damage_df['damage:flood/water'].sum()
rubble_weight = (len(damage_df)-damage_df["damage:rubble"].sum())/damage_df["damage:rubble"].sum()
misc_weight = (len(damage_df)-damage_df["damage:misc"].sum())/damage_df["damage:misc"].sum()
pos_weight = torch.as_tensor([flood_weight, rubble_weight, misc_weight], dtype=torch.float).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(net.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.9)

Finally, we're going to do the actual training.

In [ ]:
def train_model(net, train_loader, test_loader, criterion, optimizer, scheduler, logs_path, model_name, 
                starting_epoch=0, additional_epochs=10, print_every_num_batches=100):
    model_name_base = f'resnet50-{model_name}'+'.ep{}.pth'
    writer = SummaryWriter(logs_path)
    checkpoints_path = logs_path/'checkpoints'
    checkpoints_path.mkdir(parents=True, exist_ok=True)
    if starting_epoch > 0:
        starting_epoch_string = str(starting_epoch).zfill(3)
        model_load_path = checkpoints_path/model_name_base.format(starting_epoch_string)
        net.load_state_dict(torch.load(model_load_path))
    for epoch in range(starting_epoch, starting_epoch+additional_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        running_epoch_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data['image'].to(device)
            labels = data['label'].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            running_epoch_loss += loss.item()
            if (i+1) % print_every_num_batches == 0:    # print every 10 mini-batches
                print(f'[epoch {epoch+1}, batch {i +1} ] average loss: {running_loss/print_every_num_batches}')
                running_loss = 0.0

        average_epoch_loss = running_epoch_loss/(i+1)
        writer.add_scalar('Loss/epoch_avg/train', average_epoch_loss, epoch)
        print(f'[epoch {epoch+1}] average training epoch loss: {average_epoch_loss}')
        writer.add_scalar('LR/rate', scheduler.get_last_lr()[0], epoch)
        scheduler.step()
        running_epoch_loss = 0.0
        print("Getting epoch test loss...")
        for i, data in enumerate(test_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data['image'].to(device)
            labels = data['label'].to(device)

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            running_epoch_loss += loss.item()

        average_epoch_loss = running_epoch_loss/(i+1)
        writer.add_scalar('Loss/epoch_avg/test', average_epoch_loss, epoch)
        print(f'[epoch {epoch+1}] average test epoch loss: {average_epoch_loss}')
        epoch_string = str(epoch+1).zfill(3)
        model_save_path = checkpoints_path/model_name_base.format(epoch_string)
        torch.save(net.state_dict(), model_save_path)

    print('Finished Training')
    writer.close()

In [ ]:
outputs = pathlib.Path('outputs')
outputs.mkdir(exist_ok=True, parents=True)

model_name = 'damage_model'

# train damage model
train_model(net, damage_train_loader, damage_test_loader, criterion, optimizer, scheduler, outputs, model_name, 
                starting_epoch=0, additional_epochs=5, print_every_num_batches=100)

### Exercise
Using your own dataset, set up your neural network so that it is ready to train. Leave it training overnight (it will take a *long* time for it to run). Make sure there are no obvious errors in your code. 

## Performance evaluation

Now that we have a fully trained CNN, we would like to see how well it performs on unseen data. Let's first take a sample of testing images and displaying their ground truth.

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(damage_test_loader)
single_iter = next(dataiter)
images = single_iter['image']
labels = single_iter['label']

In [ ]:
# print images and ground truth
imshow(utils.make_grid(images))
print('GroundTruth: \n', ' '.join('%5s' % labels[j].cpu() for j in range(len(images))))

Then, we can load the saved model and make some predictions on the images above.
Note that `True==1` and `False==0`

In [ ]:
net.load_state_dict(torch.load('outputs/checkpoints/resnet50-damage_model.ep004.pth'))

outputs = net(images.cuda())
_, predicted = torch.max(outputs, 1) #gets the index of the max prediction, 0 = flood, 1 = rubble, 2 = misc

print('Predicted: ', ' '.join('%5s' % predicted[j].cpu()
                              for j in range(len(images))))

Now, we can look at how the trained network performs on the whole testing set.

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in damage_test_loader:
        images = data['image'].cuda()
        _, labels = torch.max(data['label'].cuda(),1)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

In addition, we can look at the performance of the model on each class of ```damage:flood/water: True``` and ```damage:flood/water: False```.

In [ ]:
truth_labels = []
predicted_labels = []
with torch.no_grad():
    for data in damage_test_loader:
        images = data['image'].cuda()
        _, labels = torch.max(data['label'].cuda(),1)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        truth_labels.append(labels.cpu())
        predicted_labels.append(predicted.cpu())
truth_labels = np.concatenate([x.numpy() for x in truth_labels])
predicted_labels = np.concatenate([x.numpy() for x in predicted_labels])

In [ ]:
import sklearn.metrics
confusion_matrix = sklearn.metrics.confusion_matrix(truth_labels, predicted_labels, labels=[0,1,2])
disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix, display_labels=['flood', 'rubble', 'misc'])
disp.plot()

In [ ]:
# visualize only flood vs no flood (rubble/misc)
confusion_matrix = sklearn.metrics.multilabel_confusion_matrix(truth_labels, predicted_labels, labels=[0,1,2])
                    # outputs 3 matrices (one for each class of 'flood', 'rubble', 'misc'), with format
                    # [[True Negatives (TN), False Positives (FP)],
                    # [False Negatives (FN), True Positives (TP)]]
                    # calculated using one-vs-rest label binarization
# use index 0 to get the confusion matrix for the flood vs no-flood counts
disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix[0], display_labels=['no flood', 'flood'])                    
disp.plot()

## Exercise
Display the performance of your neural network. Is the overall accuracy as good as you expected? What about the class accuracy? Does it perform better on one class than another? 

# Bias and ethics in machine learning
While we wait for our neural networks to train, let's take a step back and think about what it is that we're trying to do. AI actually provides a very timely and rich context in which we can discuss questions of ethics, equity and justice. 

Background material:
Designing AI with Justice (this will be our main piece), Costanza-Chock: https://www.publicbooks.org/designing-ai-with-justice/

Can we make artificial intelligence ethical?, Schwarzman: https://www.washingtonpost.com/opinions/2019/01/23/can-we-make-artificial-intelligence-ethical/

The Apple Card Didn't 'See' Gender—and That's the Problem, Knight: https://www.wired.com/story/the-apple-card-didnt-see-genderand-thats-the-problem/

Color film was built for white people. Here's what it did to dark skin, Vox: https://www.youtube.com/watch?v=d16LNHIEJzs

## Science as a social process
Something that might seem a bit strange and perhaps even a bit uncomfortable is the idea of science as a social construction. There's a couple of things to note here:
- **Not everyone subscribes to this view of science.** There are a lot of more classical schools of thought that hold science as a sort of bastion of everything that is objective.
- **Just because science is socially constructed does not mean that it does not exist.** Can you think of something else that exists even though it is a social construct?

How are some ways that the social construction of science can present itself?
- Who funds research and why? Under what constraints?
- For whom is the product of research directed to?
- Who creates the products?
- Who holds (or has historically held) a position of position of power in society?
- To what end is knowledge being generated?

**How are some ways that human bias can seep into machine learning algorithms?**

**How does this apply to the aerial imagery we are working with?**

**What do we do about it? Should we just give up on creating these models?**

### Exercise
Take a critical look at both the LADI dataset and the techniques that we are applying to it. As a group, answer the following questions:
- How might human bias be present in the LADI dataset? Think about both the imagery and the labels, as well as how these data were gathered?
- How might human bias be present in the machine learning techniques we apply to these imagery?
- If we were to build a classifier that can predict your choice of target, how might the problems outlined above yield issues in equity?
- To what extent can these issues be resolved with the dataset and machine learning techniques as they exist?
- If you could completely redo the process of arriving at the LADI dataset and doing your analysis, what would you do differently?

Be prepared to present on these at the end of the discussion period.